<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/Twitter_Topic_Category_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Sources:
# https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/
# https://developer.twitter.com/en/docs/twitter-api/annotations/overview
# For command line stuff use ! before the command for it to work.

In [12]:
# pip install tweepy

In [20]:
pip install --upgrade twarc

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 97 kB 5.3 MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=803f277534217983df9480d87b937ba4a78bdfc4d5afbcc0d89af8b083fc75c8
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj
  Attempting uninstall: humanize
    Found existing installation: humanize 0.5.1
    Uninstalling humanize-0.5.1:
      Successfully uninstalled humanize-0.5.1


In [14]:
# !twarc2 configure

In [15]:
# Configuration keys -- keep just in case.
# import tweepy

# auth = tweepy.OAuthHandler("09bm0NBwQ66dep7yzsH5U13XP", "HWZ3DOoq7WZfOcY5JWmTImOXybMcVLPUkyOPfCEtVbHRray3UM" )
# # Bearer: Do not delete: AAAAAAAAAAAAAAAAAAAAAB6MZQEAAAAA83pXM5U%2F%2Fxx3RMptSJU3pdBaw54%3DLWhdgT2cyplt2UkFk1H5wUvaIXwpCYZMAXTTCbjEMcdHbRskHO
# auth.set_access_token("56585069-1vaful5MiHoyHT0SHvQVH0qDRZ5WwMv2h4muXe5Az", "ryy4e02nJDhxldrA0437FErbm1IsdF4wmetYpDD1Xmh14")

# api = tweepy.API(auth)
# public_tweets = api.home_timeline()
# for tweet in public_tweets:
#     print(tweet.text)

In [16]:
# print(public_tweets[0])

In [17]:
# # Source: https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json
# import json
# print(json.dumps(public_tweets[0]._json, indent=4, sort_keys=True))

In [146]:
# Search Twitter for anything and it will output as a JSONL file (multiple lines of a JSON)
# Source: https://twittercommunity.com/t/how-to-save-a-json-file-using-twitter-api-v2-and-the-idle-of-python/157289/2
# Source: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query

!twarc2 search --archive --limit 5000 "olympics is:verified -is:reply -is:retweet" results.jsonl
# !twarc2 search --archive --limit 5000 "olympics (is:verified AND (-is:retweet OR -is:reply))" results.jsonl
# !twarc2 search --archive --start-time "2022-02-17" --end-time "2022-02-18" results.jsonl
# !twarc2 search --archive --start-time "2022-02-01" --end-time "2022-02-18" "from:tacobell" results.jsonl
# !twarc2 search --start-time 2022-02-17 --end-time 2022-02-18 tweets.jsonl

# !twarc2 timeline deray tweets.jsonl
# !twarc2 sample sample.jsonl
# !curl --location --request GET 'https://api.twitter.com/2/tweets/1212092628029698048?tweet.fields=context_annotations,entities' --header 'Authorization: Bearer $BEARER_TOKEN'

  0% 0/502769344543 [00:00<?, ?it/s]
⚡ There were errors processing your request: Reference to invalid operator 'exclude'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 22), Reference to invalid field 'exclude' (at position 22)


In [145]:
# Source: https://stackoverflow.com/questions/50475635/loading-jsonl-file-as-json-objects
# Source: https://www.geeksforgeeks.org/python-convert-string-dictionary-to-dictionary/
# Source: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query

import json

# Open the jsonl file and put it into a list
with open('./results.jsonl', 'r') as json_file:
    json_list = list(json_file)

# I'm guessing each json line in jsonl contains 100 tweets
print("Length of json_list:", len(json_list))

labeled_tweet_total_count = 0
# For every 100 tweets...
for l in range(len(json_list)):
  # Unwrap the list of dictionaries via json.loads()
  all_tweets_data = json.loads(json_list[l])["data"]
  # total_tweets_found.append(len(all_tweets_data))

  print(all_tweets_data[0])
  # For each tweet in the result...
  for i in range(len(all_tweets_data)):
    try:
      # Increment labeled_tweet_total_count
      labeled_tweet_total_count += 1
      # Grab the context annotations dictionary only
      dict_of_context_annotations = all_tweets_data[i]["context_annotations"]
      # For each context annotation...
      for j in range(len(dict_of_context_annotations)):
        # Grab the domain id (label)
        print("Domain_ID:", dict_of_context_annotations[j]["domain"]["id"])
        print("Domain:", dict_of_context_annotations[j]["domain"]["name"])
        # Grab the entity's name (label #2)
        print("Entity_ID:", dict_of_context_annotations[j]["entity"]["id"])
        print("Entity:", dict_of_context_annotations[j]["entity"]["name"])
    except:
      continue;

print("Total Tweets Found:", labeled_tweet_total_count)

# Loop through all 6 here.
# dict_of_context_annotations = json.loads(json_list[0])["data"][0]["context_annotations"]
# print("Number of categories", len(dict_of_context_annotations))
# for i in range(len(dict_of_context_annotations)):
#   print("Domain", dict_of_context_annotations[i]["domain"]["id"])
#   print("Entity", dict_of_context_annotations[i]["entity"]["name"])

Streaming output truncated to the last 5000 lines.
Entity: 2022 Beijing Winter Olympics
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1407828664981458949
Entity: Tokyo 2020 Summer Olympics (Limited Topics)
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1284174028442230785
Entity: Winter Olympics
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1473298560514883593
Entity: 2022 Beijing Winter Olympics
Domain_ID: 26
Domain: Sports League
Entity_ID: 1473298560514883593
Entity: 2022 Beijing Winter Olympics
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1284174028442230785
Entity: Winter Olympics
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1473298560514883593
Entity: 2022 Beijing Winter Olympics
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1473300899002863616
Entity: 2022 Beijing Winter Olympics - Skiing Cross-Country
Domain_ID: 6
Domain: Sports Event
Entity_ID: 1485616404371959808
Entity: 2022 Beijing Winter Olympics - USA Olympic Team
Domain_ID: 10
Domain: Person
Entity_ID: 951899009542627329
Enti

In [144]:
# Pretty print the json dictionary object
pretty_print = json.loads(json_list[0])
print(json.dumps(pretty_print, indent=4, sort_keys=True))

# for json_str in json_list[0]:
#     result = json.loads(json_str)
#     print(f"result: {result}")
#     print(isinstance(result, dict))

{
    "__twarc": {
        "retrieved_at": "2022-02-24T02:01:58+00:00",
        "url": "https://api.twitter.com/2/tweets/search/all?expansions=author_id%2Cin_reply_to_user_id%2Creferenced_tweets.id%2Creferenced_tweets.id.author_id%2Centities.mentions.username%2Cattachments.poll_ids%2Cattachments.media_keys%2Cgeo.place_id&tweet.fields=attachments%2Cauthor_id%2Ccontext_annotations%2Cconversation_id%2Ccreated_at%2Centities%2Cgeo%2Cid%2Cin_reply_to_user_id%2Clang%2Cpublic_metrics%2Ctext%2Cpossibly_sensitive%2Creferenced_tweets%2Creply_settings%2Csource%2Cwithheld&user.fields=created_at%2Cdescription%2Centities%2Cid%2Clocation%2Cname%2Cpinned_tweet_id%2Cprofile_image_url%2Cprotected%2Cpublic_metrics%2Curl%2Cusername%2Cverified%2Cwithheld&media.fields=alt_text%2Cduration_ms%2Cheight%2Cmedia_key%2Cpreview_image_url%2Ctype%2Curl%2Cwidth%2Cpublic_metrics&poll.fields=duration_minutes%2Cend_datetime%2Cid%2Coptions%2Cvoting_status&place.fields=contained_within%2Ccountry%2Ccountry_code%2Cfull_name%